In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, when
spark = SparkSession.builder.appName('transformations').getOrCreate()
data = [(1, 30), (2, 25), (3, 35), (4, 40), (5, 29)]
df = spark.createDataFrame(data, schema= ['a', 'b'])
filtered_df = df.filter(col('b') > 30)
filtered_df.show()
with_df = df.withColumn('c', when(col('b') > 30, '1').otherwise('0'))
with_df.show()
spark.stop()

24/08/14 18:09:24 WARN Utils: Your hostname, DESKTOP-2J74AJH resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
24/08/14 18:09:24 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/08/14 18:09:24 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/08/14 18:09:25 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


+---+---+
|  a|  b|
+---+---+
|  3| 35|
|  4| 40|
+---+---+

+---+---+---+
|  a|  b|  c|
+---+---+---+
|  1| 30|  0|
|  2| 25|  0|
|  3| 35|  1|
|  4| 40|  1|
|  5| 29|  0|
+---+---+---+



In [4]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('display').getOrCreate()
df = spark.read.csv('data.csv', header = True)
print(df.count())
df.show()
spark.stop()

3
+---+---+
|  a|  b|
+---+---+
|  1|  2|
|  3|  4|
|  5|  6|
+---+---+



In [6]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, sum, min, max
spark = SparkSession.builder.appName('aggregate').getOrCreate()
df = spark.read.csv('data.csv', header = True)
summ = df.agg(sum('b'))
summ.show()
minn = df.agg(min('b'))
minn.show()
maxx = df.agg(max('b'))
maxx.show()
avg = df.agg(avg('b'))
avg.show()
spark.stop()

+------+
|sum(b)|
+------+
|  12.0|
+------+

+------+
|min(b)|
+------+
|     2|
+------+

+------+
|max(b)|
+------+
|     6|
+------+

+------+
|avg(b)|
+------+
|   4.0|
+------+



In [8]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('write').getOrCreate()
data = [(1, ), (2, ), (3, ), (4, )]
df = spark.createDataFrame(data, schema= ['no'])
df.write.format('csv').mode('overwrite').option('header', 'true').save('./data1.csv')
spark.stop()

In [6]:
from pyspark import SparkContext
from pyspark import SparkConf
from collections import Counter
sc = SparkContext.getOrCreate(SparkConf().setMaster('local[*]'))
line = sc.textFile('text.txt')
count = line.flatMap(lambda x: x.split()).collect()
c = Counter(count)
print(c)
sc.stop()

24/08/18 13:52:13 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


['this is a test. this is a.']


['this', 'is', 'a', 'test.', 'this', 'is', 'a.']
Counter({'this': 2, 'is': 2, 'a': 1, 'test.': 1, 'a.': 1})


In [24]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as f
spark = SparkSession.builder.appName('wordCount').getOrCreate()
df = spark.read.text('text.txt')
df.show()
# f.split just splits the row value into a list
# f.explode seperates the list elements into multiple rows
df = df.withColumn('word', f.explode(f.split(f.col('value'), ' '))).groupBy('word').count()
df.show()
spark.stop()

+--------------------+
|               value|
+--------------------+
|this is a test. t...|
+--------------------+

+-----+-----+
| word|count|
+-----+-----+
|test.|    1|
|   is|    2|
|    a|    1|
| this|    2|
|   a.|    1|
+-----+-----+



In [10]:
from pyspark import SparkContext, SparkConf

spark = SparkContext.getOrCreate(SparkConf().setMaster('local[*]'))
rdd = spark.parallelize([['abc'], ['efg']])
print(rdd.collect())
a = rdd.flatMap(lambda x: x).collect()
print(a)

[['abc'], ['efg']]
['abc', 'efg']
